In [7]:
import geopandas as gpd
import pandas as pd
import folium

# 0) define your shapefile layers and join columns
layers = {
    'States': {
        'shp_path': '../vg5000_12-31.gk3.shape.ebenen/vg5000_ebenen_1231/VG5000_LAN.shp',
        'admin_col': 'GEN',
        'issues_col': 'state'
    },
    'Districts': {
        'shp_path': '../vg5000_12-31.gk3.shape.ebenen/vg5000_ebenen_1231/VG5000_KRS.shp',
        'admin_col': 'GEN',
        'issues_col': 'district'
    },
    'Municipalities': {
        'shp_path': '../vg5000_12-31.gk3.shape.ebenen/vg5000_ebenen_1231/VG5000_GEM.shp',
        'admin_col': 'GEN',
        'issues_col': 'municipality'
    },
    # add more layers (RBZ, LI, etc.) here if you like
}

# 1) load your issues data
issues_df = pd.read_csv('../data/challenge_2/complete_issues_data.csv')

# 2) build map with a single base-layer
m = folium.Map(
    location=[51.0, 10.0],
    zoom_start=6,
    tiles='OpenStreetMap',
    control_scale=True
)

# 3) for each admin-level, merge counts & add as a togglable overlay
for layer_name, params in layers.items():
    # load & reproject
    gdf = gpd.read_file(params['shp_path']).to_crs("EPSG:4326")

    # count & merge
    count_df = (
        issues_df
        .groupby(params['issues_col'])
        .size()
        .reset_index(name='issue_count')
    )
    gdf = gdf.merge(
        count_df,
        left_on=params['admin_col'],
        right_on=params['issues_col'],
        how='left'
    )

    # clean
    gdf['issue_count'] = gdf['issue_count'].fillna(0).astype(int)
    gdf = gdf[[params['admin_col'], 'issue_count', 'geometry']]

    # choropleth overlay
    folium.Choropleth(
        geo_data=gdf.to_json(),
        name=layer_name,
        data=gdf,
        columns=[params['admin_col'], 'issue_count'],
        key_on=f'feature.properties.{params["admin_col"]}',
        fill_color='YlOrRd',
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name='Number of Issues',
        highlight=True
    ).add_to(m)

    # transparent GeoJson for tooltips
    folium.GeoJson(
        gdf,
        name=f'{layer_name} Info',
        style_function=lambda feat: {
            'fillColor': 'transparent',
            'color': 'black',
            'weight': 0.5
        },
        tooltip=folium.GeoJsonTooltip(
            fields=[params['admin_col'], 'issue_count'],
            aliases=[f'{layer_name}:', 'Issues:'],
            localize=True
        )
    ).add_to(m)

# 4) final layer control (only your overlays will appear)
folium.LayerControl(collapsed=False, position='topright').add_to(m)

# 5) save
m.save('germany_issues_admin_levels.html')
